In [ ]:
import csv
import os

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from model import get_model

from detection.engine import train_one_epoch, evaluate
from dataset import dataloader, Preprocess
from infrence import output_prediction, detect_and_recognize

In [2]:
epoches = 200
batches = 20
num_classes = 11

In [3]:
train_path = "data/train"
train_json = "data/train.json"
valid_path = "data/valid"
valid_json = "data/valid.json"
test_path = "data/test"
trial_path = f"output/trial_{len(os.listdir("output")) + 1}"
os.makedirs(trial_path)
test_json = trial_path + "/pred.json"
train_loader = dataloader(train_path, 
                         train_json, 
                         preprocess=Preprocess["train"], 
                         batch_size=batches, 
                         shuffle=True)
valid_loader = dataloader(valid_path, 
                         valid_json, 
                         preprocess=Preprocess["valid"], 
                         batch_size=1, 
                         shuffle=True)
trial_path

'output/trial_9'

In [ ]:
csv_path = trial_path + "/result.csv"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = get_model()
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01)#, momentum=0.9, weight_decay=0.0005
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        header = ['Epoch', 'Train_Loss', 'mAP']
        header += [f'AP_number_{i}' for i in range(num_classes-1)]
        writer.writerow(header)

In [5]:
best_map = 0.0
for epoch in range(epoches):
    train_stats = train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=800)
    train_loss = train_stats.meters['loss'].global_avg
    #lr_scheduler.step()

    coco_eval = evaluate(model, valid_loader, device=device)
    mAP = coco_eval.coco_eval['bbox'].stats[0].item()

    precision = coco_eval.coco_eval['bbox'].eval['precision']
    num_classes = precision.shape[2]
    per_class_ap = []
    for k in range(num_classes):
        precision_k = precision[:, :, k, 0, 2]
        valid_precision = precision_k[precision_k > -1]
        if valid_precision.size > 0:
            ap = valid_precision.mean().item()
        else:
            ap = float('nan')
        per_class_ap.append(ap)

    if mAP > best_map:
        best_map = mAP
        torch.save(model.state_dict(), trial_path + f"/model_{epoch}_{mAP}.pth")


    with open(csv_path, 'a', newline='') as f:
        writer = csv.writer(f)
        row = [epoch, train_loss, mAP] + per_class_ap
        writer.writerow(row)

        lr_scheduler.step()
        print(f"[Epoch {epoch}] Train Loss: {train_loss:.4f}, mAP: {mAP:.4f}")

c:\Users\CVML_5\Desktop\ziyang\ALL_HW\DLCV_HW2\detection\engine.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [   0/1504]  eta: 0:36:01  lr: 0.000020  loss: 2.7596 (2.7596)  loss_classifier: 2.4612 (2.4612)  loss_box_reg: 0.2241 (0.2241)  loss_objectness: 0.0609 (0.0609)  loss_rpn_box_reg: 0.0133 (0.0133)  time: 1.4369  data: 0.0790  max mem: 16958
Epoch: [0]  [ 400/1504]  eta: 0:14:44  lr: 0.004016  loss: 0.4923 (0.6787)  loss_classifier: 0.2412 (0.4088)  loss_box_reg: 0.2394 (0.2393)  loss_objectness: 0.0064 (0.0184)  loss_rpn_box_reg: 0.0103 (0.0122)  time: 0.7997  data: 0.0620  max mem: 16959
Epoch: [0]  [ 800/1504]  eta: 0:09:22  lr: 0.008012  loss: 0.4255 (0.5855)  loss_classifier: 0.2165 (0.3272)  loss_box_reg: 0.1996 (0.2348)  loss_objectness: 0.0049 (0.0128)  loss_rpn_box_reg: 0.0079 (0.0107)  time: 0.7961  data: 0.0604  max mem: 16959
Epoch: [0]  [1200/1504]  eta: 0:04:03  lr: 0.010000  loss: 0.3216 (0.5165)  loss_classifier: 0.1420 (0.2769)  loss_box_reg: 0.1741 (0.2191)  loss_objectness: 0.0043 (0.0105)  loss_rpn_box_reg: 0.0067 (0.0100)  time: 0.9604  data: 0.1521  max

KeyboardInterrupt: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_model(num_classes=11)
model.load_state_dict(torch.load("best_model.pth"))

image_folder = "path/to/your/images"

coco_results, recog_results = generate_predictions(model, image_folder, device)
output_prediction(coco_results, recog_results)
